<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:

As 1blue3brown says, a neural network can be thought of as a(n extremely complicated) function. The input layer just contains the inputs to that function/network.

### Hidden Layer:

A hidden layer takes in an input from the previous layer, puts it through a set of functions (weights for each input, a bias value, and a sigmoid or other normalization function), gets an output, and sends it to the next layer.

### Output Layer:

The output layer is the final layer in the network; if we think of the network as a function, it's the function's output. In other words, it contains the values that are used to make the final judgment.

### Neuron:

A nueron is essentially a function--a set of weights for each input, a bias, an a normalization function. It takes inputs from all neurons in the previous layer, calculates an output, and sends that output on to the following layer.

### Weight:

Each neuron contains a weight--which is just a number--for each of its inputs. The input gets multiplied by the weight. This means that, as the weight gets farther from 0, that input's importance to the neuron's output also increases.

### Activation Function:

After the neuron takes in its inputs and calculates an output, that output is then run through the activation function. The purpose of the activation function is to determine how much "signal" is passed on to the subsequent layer. For instance, if the activation function is sigmoid, then the output is normalized to be between 0 and 1; if it's relu, then the activation function sets any value lower than 0 to be 0. All neurons in a given layer typically have the same activation function.

### Node Map:

A node map shows the basic structure of a neural network--it displays each layer and how each layer interacts with the other layers.

### Perceptron:

A perceptron is the most basic type of neural network, with only two layers: an input layer, and an output layer that contains a single neuron.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

You start by taking the inputs--the basis of your prediction--and putting them into the input layer (each input gets assigned to a single neuron). *Every single input* is then sent to *all* the neurons in the following layer. Each of those neurons is a function that contains weights, a bias, and an activation function. Here's how that function works:

1) The neuron. has one weight for each of its inputs. You multiply the input by its weight.

2) After multiplying the inputs by the weights, you then add the bias to the result.

3) Finally, you take the result of step 2 and plug it into the activation function.

4) That gives you an activation value, which is the neuron's output. The output it sent on to the next layer.

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [2]:
##### Your Code Here #####

# Let's separate into an X input and y output.

X = df[['x1', 'x2']]
y = df['y']
X

,x1,x2
0,0,0
1,1,0
2,0,1
3,1,1


In [89]:
# A NAND gate returns 0 when all inputs are 1, and returns 1 otherwise.
# I think I'll have each weight be 1, and the bias be 1 less than the number of columns.
# Then, when I subtract the bias, the result will only be over 0 if all inputs are 1.

def perceptron_nand_gate(X):

    weights = np.ones(len(X.columns))
    bias = 0 - (len(X.columns) - 1)
    weighted_sum = np.dot(X, weights) + bias
    
    def activate(x):
        return np.piecewise(x, [x <= 0, x > 0], [1, 0])
    
    return activate(weighted_sum)

In [90]:
perceptron_nand_gate(X)

# success!

array([1., 1., 1., 0.])

## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [91]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [165]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(diabetes)

In [166]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

X_train = train[feats]
X_test = test[feats]
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
8,2,197,70,45,543,30.5,0.158,53
435,0,141,0,0,0,42.4,0.205,29
398,3,82,70,0,0,21.1,0.389,25
75,1,0,48,20,0,24.7,0.140,22
127,1,118,58,36,94,33.3,0.261,23


In [167]:
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.transform(X_test)
X_train_norm[:5]

array([[3.40835571e-03, 3.35723038e-01, 1.19292450e-01, 7.66880035e-02,
        9.25368576e-01, 5.19774246e-02, 2.69260101e-04, 9.03214264e-02],
       [0.00000000e+00, 9.39586631e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.82542363e-01, 1.36606567e-03, 1.93248314e-01],
       [2.66171731e-02, 7.27536065e-01, 6.21067373e-01, 0.00000000e+00,
        0.00000000e+00, 1.87207451e-01, 3.45136011e-03, 2.21809776e-01],
       [1.62240430e-02, 0.00000000e+00, 7.78754064e-01, 3.24480860e-01,
        0.00000000e+00, 4.00733862e-01, 2.27136602e-03, 3.56928946e-01],
       [5.86624207e-03, 6.92216565e-01, 3.40242040e-01, 2.11184715e-01,
        5.51426755e-01, 1.95345861e-01, 1.53108918e-03, 1.34923568e-01]])

In [168]:
y_train = train['Outcome']
y_test = test['Outcome']

In [169]:
X_train_norm.shape

(576, 8)

In [170]:
y_train.shape

(576,)

In [171]:
##### Update this Class #####

class Perceptron(object):
    
    def __init__(self, niter=10, rate=0.01):
        self.niter = niter
        self.rate = rate
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        sx = self.__sigmoid(x)
        return sx * (1-sx)

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """

        if X.shape[0] != y.shape[0]:
            raise ValueError('Input and output must have the same number of samples')
    
        # Randomly Initialize Weights
        self.weights = 2 * np.random.random((X.shape[1]+1,)) - 1
        # the bias value is included in the weights array

        for _ in range(self.niter):
            
            # Weighted sum of inputs / weights
            weighted_sum = np.dot(X, self.weights[1:]) + self.weights[0]
            # Activate!
            activated_output = sigmoid(weighted_sum)
            # Calc error
            error = np.array(y) - activated_output
            # Update the Weights
            adjusted = error * self.__sigmoid_derivative(activated_output)
            self.weights[0] += np.mean(adjusted)
            self.weights[1:] += np.dot(X.T, adjusted)
            
        return self

    def predict_proba(self, X):
        """Return predicted probabilities"""
        
        weighted_sum = np.dot(X, self.weights[1:]) + self.weights[0]
        return sigmoid(weighted_sum)
    
    def predict(self, X):
        """Return class label after unit step"""
        
        y_pred = self.predict_proba(X)    
        return np.where(y_pred < 0.5, 0, 1)

In [178]:
network = Perceptron(niter=10000)
network.fit(X_train_norm, y_train)
y_pred = network.predict(X_test)

In [179]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7291666666666666

In [180]:
y_pred

# I sometimes get all zeroes in my results...it takes multiple instantiations of the model
# to get decent results like below. Is that bad?

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?